In [72]:
# Note to import from .py files, must follow structure
# from <.py filename excluding '.py'> import <class name>
# Optionslam creds: aspringfastlaner Options2018

# Importing necessary models
import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
from pandas import ExcelWriter
import matplotlib.pyplot as plt
import os
import seaborn as sns
import matplotlib.dates as dates
# import matplotlib.ticker as ticker
from lxml import html
import requests
import webbrowser
from bs4 import BeautifulSoup as bs
import json
import csv
import sched, time
import pandas_datareader as datareader
from pandas_datareader.data import Options
from alpha_vantage.timeseries import TimeSeries
ts = TimeSeries(key='5HZEUI5AFJB06BUK',output_format='pandas')
import matplotlib.pyplot as plt
import urllib.request as urlreq
import statsmodels.formula.api as sm

from pandas_datareader.data import Options
import py_vollib
from py_vollib.black_scholes_merton.implied_volatility import *
from py_vollib.black_scholes_merton.greeks.analytical import *

'''
Calculate the Black-Scholes implied volatility.

Parameters:	
price (float) – the Black-Scholes option price
S (float) – underlying asset price
K (float) – strike price
t (float) – time to expiration in years
r (float) – risk-free interest rate
flag (str) – ‘c’ or ‘p’ for call or put.
>>> S = 100
>>> K = 100
>>> sigma = .2
>>> r = .01
>>> flag = 'c'
>>> t = .5
>>> price = black_scholes(flag, S, K, t, r, sigma)
>>> iv = implied_volatility(price, S, K, t, r, flag)
'''
%matplotlib inline

def write_excel(filename, sheetnames, df_list):
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(filename, engine='xlsxwriter')
    for i, df in enumerate(df_list):
        
        df.to_excel(writer, sheet_name = sheetnames[i])

    # Close the Pandas Excel writer and output the Excel file.
    writer.save()
    return

# Alpha Vantage API Key
# 5HZEUI5AFJB06BUK

# ts = TimeSeries(key='5HZEUI5AFJB06BUK', output_format='pandas')
# data, meta_data = ts.get_intraday(symbol='MSFT',interval='1min', outputsize='full')
# data['close'].plot()
# plt.title('Intraday Times Series for the MSFT stock (1 min)')
# For intraday
# https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=MSFT&interval=1min&apikey=d5HZEUI5AFJB06BUK&datatype=csv

# For daily
# https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=5HZEUI5AFJB06BUK&datatype=csv

In [2]:
'''
Pulling S&P 500 Names
'''

def pull_sp500_list():
    site = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    table = soup.find_all('table')[0]

    tickers = []
    names = []
    gics = []

    # Looping through the soup lxml text table format
    # and splitting each row as a individual string
    # and parsing string to retrieve the date,
    # open, and close information.
    i = 1
    for row in table.find_all('tr'):
        if i == 1:
            i += 1
            continue
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        # row_items is parsed string for each current row where each
        ticker = individual_row[1].split('">')[-1].split('<')[0]
        tickers.append(ticker)
        name = individual_row[2].split('">')[-1].split('<')[0]
        names.append(name)
        gic = individual_row[4].split('>')[1].split('<')[0]
        gics.append(gic)

    sp500 = pd.DataFrame({'Name': names, 'GIC': gics}, index = tickers)
    sp500.index.name = 'Tickers'
    return sp500

#nasdaq = pd.read_csv('http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NASDAQ&render=download', index_col = 0)[['Name','LastSale','IPOyear','Sector']]
#nyse = pd.read_csv('http://www.nasdaq.com/screening/companies-by-industry.aspx?exchange=NYSE&render=download', index_col = 0)[['Name','LastSale','IPOyear','Sector']]
#us_stocks = pd.concat([nyse,nasdaq], axis = 0).drop_duplicates()
#us_stocks = us_stocks[us_stocks['LastSale'] != 'n/a']
#us_fundamentals = get_fundas([string.replace(' ','') for string in us_stocks.index.tolist()])
us_stocks = pd.read_csv('us_stocks.csv', index_col = 0)

active_stocks = pd.read_csv('active_names.csv', index_col = 0).dropna()
active_etfs = pd.read_csv('active_etfs.csv', index_col = 0).dropna()
highest_ivs = pd.read_csv('highest_iv.csv', index_col = 0).dropna()

# filtered_names = pd.read_csv('filtered_names.csv', index_col = 0).join(us_stocks, how = 'inner')[us_stocks.columns]
# filtered_names['Market Cap'] = filtered_names['Market Cap'].astype(str).str[:-1]
# filtered_names['Market Cap'] = filtered_names['Market Cap'].astype(float)
# filtered_names = filtered_names.sort_values(['Market Cap'], ascending = False)

watchlist = ['NVDA', 'FB', 'AMZN', 'NFLX', 'GOOGL', 'GOOG',
             'TSLA', 'EA', 'ATVI', 'APPL', 'MSFT', 'INTC',
             'V', 'CSCO', 'VZ', 'T', 'MA', 'ORCL', 'IBM',
             'ADBE', 'TXN', 'AVGO', 'PYPL']

In [3]:
'''
Function for pulling implied volatility from option slam for single ticker
'''

def optionslam_scrape(ticker):
    site = 'https://www.optionslam.com/earnings/stocks/' + ticker
    res = requests.get(site)
    soup = bs(requests.get(site).text, "lxml")
    soup = soup.prettify()
    earnings_dict = {'Ticker': ticker}
    
    # Check if there's weekly options
    curr7_implied = "Current 7 Day Implied Movement:"
    implied_move_weekly = "Implied Move Weekly:"
    nextearnings = "Next Earnings Date:"
    if curr7_implied not in soup:
        return 'No Weeklies'
    
    # Parsing if weekly options exist
    # Next earnings date and before or after
    earnings_start_string = "Next Earnings Date:"
    earnings_end_string = '</font>'
    raw_earnings_string = (soup.split(earnings_start_string))[1].split(earnings_end_string)[0].replace('\n','').strip()
    
    try:
        earnings_date = str((raw_earnings_string.split('<b>'))[1].split('<font size="-1">')).split("'")[1].strip()
    except:
        return 'Error Parsing'
    
    earnings_time = str(raw_earnings_string[-2:].strip()).strip()
    
    earnings_dict['Date'] = earnings_date
    earnings_dict['Earnings Time'] = earnings_time
    
    # Parsing 7 day implied move if weekly option exists
    ending_string = '<font size="-2">'
    curr_7 = (soup.split(curr7_implied))[1].split(ending_string)[0].replace('\n','').strip("").split("<td>")[-1].strip()
    earnings_dict['Current 7 Day Implied'] = curr_7
    
    # Parsing Weekly Implied move if weekly option exists
    if implied_move_weekly in soup:
        weekly_implied = (soup.split(implied_move_weekly))[1].split(ending_string)[0].replace('\n','').strip("").split("<td>")[-1].strip()
    else:
        weekly_implied = ''
    earnings_dict["Implied Move Weekly"] = weekly_implied
    
    return earnings_dict

# Looping through the soup lxml text table format
# and splitting each row as a individual string
# and parsing string to retrieve the date,
# open, and close information.

def yahoo_table_parse(raw_html_table):
    tickers = []
    call_times = []
    implied_7_day = []
    implied_weekly = []
    eps = []
    i = 1
    end_row = 10
    for row in raw_html_table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')
        row_items = individual_row[0].split('</span>')[:3]

        if i == 1:
            i += 1
            continue
        tick = row_items[0].split('data-symbol="')[1].split('"')[0]
        os_check = optionslam_scrape(tick)

        if type(os_check) == str:
            continue
        else:
            tickers.append(tick)
            call_times.append(row_items[0].split('data-symbol="')[1].split('"')[-1].replace('>',''))
            eps.append(row_items[1].split('</td>')[1].split('>')[1])
            implied_7_day.append(os_check['Current 7 Day Implied'].replace('%',''))
            implied_weekly.append(os_check['Implied Move Weekly'].replace('%',''))


    return pd.DataFrame({'Tickers': tickers, 'Call Times': call_times, 'EPS': eps,
                         'Current 7 Day Implied': implied_7_day,
                         'Implied Move Weekly': implied_weekly})


def yahoo_earnings(date):
    # Yahoo Earnings Calendar Check

    today = date.strftime('%Y-%m-%d')
    tables = []
    for i in range(6):
        yahoo_url = 'https://finance.yahoo.com/calendar/earnings?day=' + today + '&offset={}&size=100'.format(int(i*100))
        res = requests.get(yahoo_url)
        soup = bs(requests.get(yahoo_url).text, "lxml")

        try:
            table = soup.find_all('table')[0]
            tables.append(yahoo_table_parse(table))
        except:
            print('No Table')

    return pd.concat(tables,axis = 0, ignore_index = True)

'''
Functions for pulling options data from yahoo Input is a string. The output is a dataframe of the latest
data from yahoo finance tagged with the current date-time. Output columns are pull date-time,
contract name, strike, last price, bid, ask volume, open interest, and IV (in decimal form).
'''
# Function for initial querying of yahoo data
def yahoo_option_query(ticker, unix_date):
    # dt.datetime.fromtimestamp(1525996800).date()
    if unix_date == 'None':
        yahoo_query = 'https://query1.finance.yahoo.com/v7/finance/options/{0}'.format(ticker)
    else:
        yahoo_query = 'https://query1.finance.yahoo.com/v7/finance/options/{0}?date={1}'.format(ticker,str(unix_date))
        
    response = urlreq.urlopen(yahoo_query)
    data = json.loads(response.read().decode())['optionChain']['result'][0]
    
    dict_lst = []
    for key in data.keys():
        dict_lst.append(data[key])
        
    expiries = dict_lst[1]
    strikes = dict_lst[2]
    underlying = dict_lst[4]
    calls = dict_lst[5][0]['calls']
    puts = dict_lst[5][0]['puts']
    
    return (expiries, strikes, underlying, calls, puts)

# Function for creating dataframe for options contracts for a specific maturity date
def create_contract_df(option_dict_lst, strikes):
    df = pd.DataFrame(columns = ['lastPrice','volume','openInterest','bid','ask','mid','impliedVolatility','expiration'],
                      index = strikes)
    for contract in option_dict_lst:
        for col in df.columns:
            if col == 'expiration':
                df.loc[contract['strike'], col] = (dt.datetime.fromtimestamp(contract['expiration']) - dt.datetime.today()).days
            elif col == 'mid':
                df.loc[contract['strike'], col] = (contract['ask'] + contract['bid'])/2
            else:
                df.loc[contract['strike'], col] = contract[col]
    return df.dropna()

# Function for creating straddle view of options for a specific date
def option_chain(calls, puts, strikes):
    call_contracts = create_contract_df(calls, strikes)
    put_contracts = create_contract_df(puts, strikes)
    return call_contracts.join(put_contracts, how = 'inner', lsuffix='_c', rsuffix='_p')

# Function for getting full option data for a specific ticker
def pull_options(ticker):
    initial_near_contract = yahoo_option_query(ticker, 'None')
    
    expiries = initial_near_contract[0]
    options_list = [option_chain(initial_near_contract[3], initial_near_contract[4], initial_near_contract[1])]
    
    for expiry in expiries[1:]:
        next_contract = yahoo_option_query(ticker, expiry)
        options_list.append(option_chain(next_contract[3], next_contract[4], next_contract[1]))
        
    return pd.concat(options_list, axis = 0)
# ts.get_daily('AAPL')[0].tail(1)['close'][0]
'''
Function for getting all relevant earnings for a given starting week (Monday in dt.datetime(YYYY, m, d) format)
Returns a dataframe with the earnings names, implied move, price, and earnings times.
'''
def weekly_earnings_check(start_datetime, days_forward):

    start_date = start_datetime

    weekly_earnings = []
    while start_date.weekday() < days_forward:
        try:
            temp_earnings = yahoo_earnings(start_date)
            temp_earnings['Earnings Date'] = start_date
            temp_earnings['Last Close'] = 0
            for idx, row in temp_earnings.iterrows():
                temp_earnings.loc[idx, 'Last Close'] = ts.get_daily(row['Tickers'])[0].tail(1)['close'][0]
            weekly_earnings.append(temp_earnings)
            start_date = start_date + dt.timedelta(1)
        except:
            start_date = start_date + dt.timedelta(1)

    earnings_df = pd.concat(weekly_earnings,axis = 0, ignore_index = True)
    earnings_df = earnings_df[earnings_df['Last Close'] >= 30]
    earnings_df['Implied Move Weekly'] = pd.to_numeric(earnings_df['Implied Move Weekly'])
    #earnings_df = earnings_df[earnings_df['Call Times'] != '-']
    earnings_df['Lower Bound'] = np.round(earnings_df['Last Close']*(1 - earnings_df['Implied Move Weekly']/100),2)
    earnings_df = earnings_df.sort_values(['Earnings Date','Call Times'])
    
    return earnings_df

def stock_earnings(tick):
    yahoo_url = 'https://finance.yahoo.com/calendar/earnings/?symbol={0}'.format(tick)
    res = requests.get(yahoo_url).text
    res = res.split('table class')[1].split('</table>')[0].split('<tbody data-reactid="')[1].split('<td class="')
    earnings_cols = list(filter(lambda x: 'data-col2' in x, res))
    estimate_cols = list(filter(lambda x: 'data-col3' in x, res))
    reported_cols = list(filter(lambda x: 'data-col4' in x, res))

    earnings_days = []
    for i in earnings_cols:
        temp_date = i.split('">')[2].split('</')[0]
        temp_date = ''.join(temp_date.split(',')[:-1])
        temp_date = dt.datetime.strptime(temp_date, '%b %d %Y')
        earnings_days.append(temp_date)

    estimate_eps = []
    for i in estimate_cols:
        try:
            temp_est = float(i.split('">')[1].split('<')[0])
        except:
            temp_est = np.nan
        estimate_eps.append(temp_est)

    reported_eps = []
    for i in reported_cols:
        try:
            temp_est = float(i.split('">')[1].split('<')[0])
        except:
            temp_est = np.nan
        reported_eps.append(temp_est)

    earnings = pd.DataFrame({'Earnings Dates': earnings_days,
                             'EPS Estimate': estimate_eps,
                             'EPS Reported': reported_eps})
    return earnings

def old_weekly_earnings_check(start_datetime, days_forward):
    
    def yahoo_earnings_raw(raw_html_table):
        tickers = []
        eps = []
        i = 1
        end_row = 10
        for row in raw_html_table.find_all('tr'):
            # Individual row stores current row item and delimits on '\n'
            individual_row = str(row).split('\n')
            row_items = individual_row[0].split('</span>')[:3]

            if i == 1:
                i += 1
                continue
            tick = row_items[0].split('data-symbol="')[1].split('"')[0]
            tickers.append(tick)


        return pd.DataFrame({'Tickers': tickers})
    
    def yahoo_earnings_old(date):
        # Yahoo Earnings Calendar Check

        today = date.strftime('%Y-%m-%d')
        tables = []
        for i in range(6):
            yahoo_url = 'https://finance.yahoo.com/calendar/earnings?day=' + today + '&offset={}&size=100'.format(int(i*100))
            res = requests.get(yahoo_url)
            soup = bs(requests.get(yahoo_url).text, "lxml")

            try:
                table = soup.find_all('table')[0]
                tables.append(yahoo_earnings_raw(table))
            except:
                print('No Table')

        return pd.concat(tables,axis = 0, ignore_index = True)
    
    start_date = start_datetime

    weekly_earnings = []
    while start_date.weekday() < days_forward:
        try: 
            temp_earnings = yahoo_earnings_old(start_date)
            temp_earnings['Earnings Date'] = start_date
            weekly_earnings.append(temp_earnings)
            start_date = start_date + dt.timedelta(1)
        except:
            start_date = start_date + dt.timedelta(1)
            continue

    earnings_df = pd.concat(weekly_earnings,axis = 0, ignore_index = True)
    #earnings_df = earnings_df[earnings_df['Call Times'] != '-']
    
    return earnings_df

    
def fundamentals(ticker):
    
    site = 'https://finance.yahoo.com/quote/{0}?p={0}'.format(ticker)

    res = requests.get(site)
    soup = bs(res.text, 'lxml')
    table = soup.find_all('table')[1]
    sum_dict = {}

    # Looping through the soup lxml text table format
    # and splitting each row as a individual string
    # and parsing string to retrieve the date,
    # open, and close information.


    for row in table.find_all('tr'):
        # Individual row stores current row item and delimits on '\n'
        individual_row = str(row).split('\n')[0]

        # row_items is parsed string for each current row where each
        # item in list is the date, open, high, low, close, and volume
        row_items = individual_row.split('<span data-reactid=')[1].split('"><!-- react-text: ')
        
        if len(row_items) > 1:
            sum_item = row_items[0].split('>')[1].split('<')[0]
            sum_value = row_items[1].split('-->')[1].split('<')[0]
        elif 'YIELD' in row_items[0]:
            try:
                temp_val = row_items[0].split('-value">')[1].split("</td>")[0]
                div_amount = float(temp_val.split(' ')[0])
                div_yield = float(temp_val.split(' ')[1].replace('(','').replace(')','').replace('%',''))

                sum_dict['Div'] = div_amount
                sum_dict['Yield'] = div_yield
            except:
                sum_dict['Div'] = np.nan
                sum_dict['Yield'] = np.nan
        elif 'Market Cap' in row_items[0]:
            sum_item = 'Market Cap (B)'
            mkt_cap = row_items[0].split('data-reactid="')[-1].split('>')[1].split('<')[0]
            mkt_cap_amount = float(mkt_cap[:-1])
            if mkt_cap[-1] == 'M':
                sum_value = mkt_cap_amount/1000
            else:
                sum_value = mkt_cap_amount
        else:
            sum_item = row_items[0].split('>')[1].split('<')[0]
            sum_value = row_items[0].split('data-reactid="')[-1].split('>')[1].split('<')[0]
        
        sum_dict[sum_item] = sum_value
    
    sum_dict['Days Since Last Earnings'] = (dt.datetime.today().date() - 
                                            stock_earnings(ticker)['Earnings Dates'][1].date()).days

    return pd.DataFrame(sum_dict, index = [ticker])

# Function to return fundametal data of a ticker list
def get_fundas(ticker_lst):
    fund_lst = []
    for tick in ticker_lst:
        try:
            fund_lst.append(fundamentals(tick))
        except:
            continue
    return pd.concat(fund_lst,axis = 0)

# Function historical data from alpha advantage
def historical_data(ticker, day_number = 252, rolling_window = 20, outsize = 'full'):
    alphavantage_link = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={0}&apikey=5HZEUI5AFJB06BUK&datatype=csv&outputsize={1}'.format(ticker, outsize)
    stockframe = pd.read_csv(alphavantage_link, index_col = 0).sort_index()[['open', 'close']]
    stockframe['daily_ret'] = np.log(stockframe['close']/stockframe['close'].shift(1))
    stockframe['intra_ret'] = np.log(stockframe['close']/stockframe['open'])
    stockframe['ovrnt_ret'] = np.log(stockframe['open']/stockframe['close'].shift(1))
    stockframe['daily_vol'] = stockframe.daily_ret.rolling(window=rolling_window,center=False).std()
    stockframe['intra_vol'] = stockframe.intra_ret.rolling(window=rolling_window,center=False).std()
    stockframe['ovrnt_vol'] = stockframe.ovrnt_ret.rolling(window=rolling_window,center=False).std()
    stockframe['daily_ann'] = stockframe.daily_vol*np.sqrt(252)
    stockframe['intra_ann'] = stockframe.intra_vol*np.sqrt((24/6.5)*252)
    stockframe['ovrnt_ann'] = stockframe.ovrnt_vol*np.sqrt((24/17.5)*252)
    stockframe['oc_diff'] = stockframe.close - stockframe.open
    stockframe['daily_dollar_vol'] = stockframe.daily_vol*stockframe.close.shift(1)
    stockframe['daily_dollar_std'] = np.abs(stockframe.oc_diff/stockframe.daily_dollar_vol)

    return stockframe.tail(day_number)

# Function for building a dataframe of volatilities
# Daily, Intraday, Overnight
def current_volatility(ticker_list, roll = 20):
    
    rows = []
    failed_tickers = []
    
    def failed_check(failed_lst,rows):
        if len(failed_lst) == 0:
            return failed_lst, rows
        else:
            new_lst = []
            new_rows = rows
            for tick in failed_lst:
                try: 
                    curr_vol = historical_data(tick, outsize = 'compact').tail(1)[['daily_ann','intra_ann','ovrnt_ann','close',
                                                                                   'daily_dollar_vol']]
                    curr_vol.index.name = 'Tickers'
                    curr_vol.index = [tick]
                    new_rows.append(curr_vol)
                except:
                    new_lst.append(tick)
            return failed_check(new_lst, rows)

    for tick in ticker_list:
        try: 
            curr_vol = historical_data(tick, outsize = 'compact').tail(1)[['daily_ann','intra_ann','ovrnt_ann','close',
                                                                           'daily_dollar_vol']]
            curr_vol.index.name = 'Tickers'
            curr_vol.index = [tick]
            rows.append(curr_vol)
        except:
            failed_tickers.append(tick)
            
    failed_lst, rows = failed_check(failed_tickers, rows)
        
    return pd.concat(rows, axis = 0)

# Function for pulling S&P500 data and calculating volatilities
def sp500_filter():
    sp500 = pull_sp500_list()
    sp500_vols = current_volatility(sp500.index.tolist())
    df = pd.concat([sp500_vols, sp500], axis = 1).dropna()
    
    return df

def get_raw_data(ticker):
    tape = Options(ticker, 'yahoo')
    data = tape.get_all_data()
    return data

# Function for pulling options for a given ticker
def option_filter(ticker, moneyness_thresh, dte_thresh):
    data = get_raw_data(ticker).reset_index()
    data['Moneyness'] = np.abs(data['Strike'] - data['Underlying_Price'])/data['Underlying_Price']
    data['DTE'] = (data['Expiry'] - dt.datetime.today()).dt.days
    data = data[['Strike', 'DTE', 'Type', 'IV', 'Vol','Open_Int', 'Moneyness', 'Root', 'Underlying_Price',
                 'Last','Bid','Ask']]
    data['Mid'] = data['Ask'] - data['Bid']

    filtered_data = data[(data['Moneyness'] <= moneyness_thresh) &
                         (data['DTE'] <= dte_thresh)].reset_index()[data.columns]
    put_ivs = filtered_data[filtered_data.Type == 'put'].pivot(index='Strike', columns='DTE', 
                                                               values='IV').dropna()
    call_ivs = filtered_data[filtered_data.Type == 'put'].pivot(index='Strike', columns='DTE', 
                                                                values='IV').dropna()
    hv_data = current_volatility([ticker])

    put_ivs['Close'] = hv_data['close'][0]
    call_ivs['Close'] = hv_data['close'][0]
    put_ivs['Daily HV'] = hv_data['daily_ann'][0]
    call_ivs['Daily HV'] = hv_data['daily_ann'][0]
    put_ivs['Intra HV'] = hv_data['intra_ann'][0]
    call_ivs['Intra HV'] = hv_data['intra_ann'][0]
    put_ivs['Overnight HV'] = hv_data['ovrnt_ann'][0]
    call_ivs['Overnight HV'] = hv_data['ovrnt_ann'][0]
    put_ivs['Daily Dollar Vol'] = hv_data['daily_dollar_vol'][0]
    call_ivs['Daily Dollar Vol'] = hv_data['daily_dollar_vol'][0]
    
    put_ivs['Moneyness'] = np.abs(put_ivs.index - put_ivs['Close'])/put_ivs['Close']
    call_ivs['Moneyness'] = np.abs(call_ivs.index - call_ivs['Close'])/call_ivs['Close']

    call_ivs.index.name = ticker + ' Call Strike'
    put_ivs.index.name = ticker + ' Put Strike'
    return call_ivs, put_ivs

def phase1_options_filter(ticker_lst, hv_day_roll, atm_distance, dte_thresh, iv_hv_thresh):
    
    pulled_options = []
    calls = {}
    puts = {}

    moneyness_thresh = atm_distance

    for tick in ticker_lst:
        try:
            call, put = option_filter(tick, moneyness_thresh, dte_thresh)
            
            call_check = call.sort_values('Moneyness').head(1)
            # Checking IV vs HV of the options chain out more than 2 weeks
            iv_chain_to_check = list(filter(lambda x: x >= hv_day_roll, 
                                            list(filter(lambda x: type(x) == int,call_check.columns.tolist()))))[0]
            
            average_difference_iv_hv = abs(call_check[iv_chain_to_check] - call_check['Intra HV']).mean()

            if average_difference_iv_hv >= iv_hv_thresh:
                calls[tick] = call
                puts[tick] = put
                pulled_options.append(tick)
        except:
            pass
        
    return pulled_options, calls, puts

def greek_calc(ticker, dte_ub, dte_lb, prem_price_use = 'Mid', delta_filter = 0.2):
    options_chain = get_raw_data(ticker).reset_index()
    options_chain = options_chain[['Strike','Expiry','Type','Last','Bid','Ask','Vol','Open_Int','IV','Underlying_Price']]
    df = options_chain
    df['DTE'] = (df['Expiry'] - dt.datetime.today()).dt.days
    df['Mid'] = (df['Ask'] + df['Bid'])/2
    df = df[(df['DTE'] <= dte_ub) & (df['DTE'] >= dte_lb)]
    df = df.reset_index()[df.columns]
    
    year = 365
    premiums = df[prem_price_use].values # 'Last' or 'Mid'
    strikes = df['Strike'].values
    time_to_expirations = df['DTE'].values
    ivs = df['IV'].values
    underlying = df['Underlying_Price'].values[0]
    types = df['Type'].values

    # Make sure nothing thows up
    assert len(premiums) == len(strikes)
    assert len(strikes) == len(time_to_expirations)

    sigmas = []
    deltas = []
    gammas = []
    thetas = []
    vegas = []
    prices = []
    for premium, strike, time_to_expiration, flag in zip(premiums, strikes, time_to_expirations, types):

        # Constants
        P = premium
        S = underlying
        K = strike
        t = time_to_expiration/float(year)
        r = 0.005 / 100
        q = 0 / 100
        try:
            sigma = implied_volatility(P, S, K, t, r, q, flag[0])
            sigmas.append(sigma)
        except:
            sigma = 0.0
            sigmas.append(sigma)

        try:
            delta = py_vollib.black_scholes_merton.greeks.analytical.delta(flag[0], S, K, t, r, sigma, q)
            deltas.append(delta)
        except:
            delta = 0.0
            deltas.append(delta)

        try:
            gamma = py_vollib.black_scholes_merton.greeks.analytical.gamma(flag[0], S, K, t, r, sigma, q)
            gammas.append(gamma)
        except:
            gamma = 0.0
            gammas.append(gamma)

        try:
            theta = py_vollib.black_scholes_merton.greeks.analytical.theta(flag[0], S, K, t, r, sigma, q)
            thetas.append(theta)
        except:
            theta = 0.0
            thetas.append(theta)

        try:
            vega = py_vollib.black_scholes_merton.greeks.analytical.vega(flag[0], S, K, t, r, sigma, q)
            vegas.append(vega)
        except:
            vega = 0.0
            vegas.append(vega)

    ivs = np.array(sigmas)
    df['Calc IV'] = ivs
    df['Delta'] = deltas
    df['Gamma'] = gammas
    df['Theta'] = thetas
    df['Vega'] = vegas
    df = df.dropna()

    expiry_filter = df.sort_values('DTE')['DTE'].drop_duplicates().values[0]

    calls = df[(abs(df['Delta']) >= delta_filter) & 
               (df['Type'] == 'call') & 
               (df['DTE'] == expiry_filter)].reset_index()[df.columns]
    puts = df[(abs(df['Delta']) >= delta_filter) & 
              (df['Type'] == 'put') & 
              (df['DTE'] == expiry_filter)].reset_index()[df.columns]
    
    return calls, puts

def price_sim(options_df, price_change, vol_change, days_change, iv_tag = 'Calc IV'):
    year = 365
    strikes = options_df['Strike'].values
    time_to_expirations = options_df['DTE'].values
    ivs = options_df[iv_tag].values
    underlying = options_df['Underlying_Price'].values[0]
    types = options_df['Type'].values

    # Tweaking changes
    prices = []
    for sigma, strike, time_to_expiration, flag in zip(ivs, strikes, time_to_expirations, types):

        # Constants
        S = underlying*(1 + price_change)
        t = max(time_to_expiration - days_change, 0)/float(year)
        K = strike
        r = 0.005 / 100
        q = 0 / 100

        try:
            sigma = sigma + vol_change
            price = py_vollib.black_scholes_merton.black_scholes_merton(flag[0], S, K, t, r, sigma, q)
            prices.append(price)
        except:
            price = 0.0
            prices.append(price)
            
    df = options_df[['Strike','Expiry','DTE','Type','Last','Bid','Ask','Mid','Underlying_Price']]
    df['Simulated Price'] = prices
    df['Price Change'] = df['Simulated Price']/df['Mid'] - 1
    df = df.dropna()
    return df

In [4]:
# active_stock_fundas = get_fundas(active_stocks[active_stocks['Last'] >= 30].index.tolist())
# active_stock_fundas = active_stock_fundas[active_stock_fundas['Market Cap (B)'] >= 5]
# tick_lst = july16_earnings['Tickers'].tolist() #
tick_lst = active_stocks[active_stocks['Last'] >= 30].head(100).index.tolist()
etfs_lst = active_etfs[active_etfs['Last'] >= 50].head(100).index.tolist()
days_since_earnings_threshold = 30
mkt_cap_thresh = 5
moneyness_thresh = 0.05
hv_roll = 20
dte_thresh = 30
sort_on = 'intra_ann'
max_options_num = 10
iv_hv_thres = 0.05

vols = current_volatility(tick_lst, roll = 22)
etfs = current_volatility(etfs_lst, roll = 22).sort_values([sort_on], ascending = False).dropna()
vols = vols.sort_values([sort_on], ascending = False).dropna()
fundas = get_fundas(vols.index.tolist())
tickers = pd.concat([vols,fundas],axis = 1)
tickers = tickers[tickers['Days Since Last Earnings'] >= 
                  days_since_earnings_threshold].sort_values([sort_on], ascending = False)

etf_ticks = etfs.sort_values([sort_on], ascending = False)

pulled_options, pulled_calls, pulled_puts = phase1_options_filter(tickers.index.tolist()[:max_options_num], 
                                                                  hv_roll, moneyness_thresh, dte_thresh,
                                                                  iv_hv_thres)

pulled_etfs, pulled_etfcalls, pulled_etfputs = phase1_options_filter(etf_ticks.index.tolist()[:max_options_num], 
                                                                     hv_roll, moneyness_thresh, dte_thresh,
                                                                     iv_hv_thres)

filtered_single_names = (tickers.transpose()[pulled_options]).transpose()
filtered_single_names = (tickers.transpose()[pulled_options]).transpose()
filtered_single_names['IV'] = np.nan
for stock in pulled_options:
    filtered_single_names.loc[stock,'IV'] = pulled_puts[stock].sort_values(['Moneyness']).head(1).iloc[:, 0:-6].iloc[:,-1].iloc[0]
    
filtered_etfs = (etf_ticks.transpose()[pulled_etfs]).transpose()
filtered_etfs = (etf_ticks.transpose()[pulled_etfs]).transpose()
filtered_etfs['IV'] = np.nan
for etf in pulled_etfs:
    filtered_etfs.loc[etf,'IV'] = pulled_etfputs[etf].sort_values(['Moneyness']).head(1).iloc[:, 0:-6].iloc[:,-1].iloc[0]

In [5]:
filtered_single_names

,daily_ann,intra_ann,ovrnt_ann,close,daily_dollar_vol,1y Target Est,Beta,Days Since Last Earnings,Div,EPS (TTM),Earnings Date,Ex-Dividend Date,Market Cap (B),PE Ratio (TTM),Yield,IV
WLL,0.521462,0.98139,0.250578,46.78,1.69632,50.52,3.56,46,NaN,-13.65,,N/A,4.258,N/A,NaN,0.549321
OLED,0.406104,0.962741,0.695005,93.95,2.5224,147.70,2.25,43,0.24,2.18,,2018-03-14,4.424,43.10,0.23,0.285774
ROKU,0.500241,0.928412,0.233704,43.3,1.41868,39.00,N/A,114,NaN,-2.24,N/A,N/A,4.399,N/A,NaN,0.589360
M,0.427123,0.773007,0.0896965,38.27,1.0106,34.31,0.50,30,1.51,5.04,,2018-03-14,11.725,7.59,4.76,0.335456
JD,0.384955,0.716645,0.181192,43.59,1.06093,48.11,1.68,38,NaN,-0.02,,N/A,62.213,N/A,NaN,0.321784
TSLA,0.465159,0.712693,0.241789,358.17,10.482,307.55,0.91,44,NaN,-13.97,,N/A,60.815,N/A,NaN,0.470953
APA,0.461591,0.705593,0.199084,41.61,1.24713,45.45,1.63,44,1,3.41,,2018-07-20,15.901,12.20,2.37,0.208016
FSLR,0.42906,0.701643,0.240979,51.06,1.45169,70.93,0.73,50,NaN,-1.59,,N/A,5.349,N/A,NaN,0.440435


In [6]:
filtered_etfs

,daily_ann,intra_ann,ovrnt_ann,close,daily_dollar_vol,IV
JDST,0.418093,0.629202,0.283579,50.57,1.279733,0.762698
FAS,0.404215,0.535545,0.214195,67.35,1.718000,0.364508
TQQQ,3.939306,0.488611,4.577870,62.74,15.725451,0.414801


In [56]:
call_ivs, put_ivs = option_filter('BABA', moneyness_thresh, dte_thresh)

test = call_ivs.iloc[:, 0:-6].iloc[:,-1]
test.columns = ['IV']
test['Strikes'] = test.index
r_sqrd = result.rsquared
slope = result.params.Strikes

In [115]:
vol2 = vols
vol2['hv_ratio'] = vol2['intra_ann']/vol2['daily_ann']
vol2 = vol2.sort_values(['hv_ratio', 'intra_ann'], ascending = False)
vol2

,daily_ann,intra_ann,ovrnt_ann,close,daily_dollar_vol,hv_ratio
CRM,0.198015,0.409468,0.229415,133.2300,1.690818,2.067867
GILD,0.159235,0.327767,0.078762,71.9700,0.721920,2.058386
PANW,0.192083,0.384597,0.176473,198.9600,2.456076,2.002246
TMUS,0.113521,0.212897,0.074488,57.1900,0.411907,1.875392
MGM,0.265309,0.492839,0.092611,31.2400,0.527292,1.857602
AAL,0.278575,0.517435,0.121217,43.3250,0.775470,1.857436
ADBE,0.160737,0.296211,0.162361,248.7400,2.571371,1.842826
WMT,0.203508,0.374671,0.084425,84.5700,1.084044,1.841060
NFLX,0.214699,0.385904,0.144388,360.2150,4.969679,1.797419
LVS,0.126539,0.226009,0.111166,78.3850,0.628926,1.786089


In [125]:
june25_earnings

,Call Times,Current 7 Day Implied,EPS,Implied Move Weekly,Tickers,Earnings Date,Last Close,Lower Bound
0,Before Market Open,0.79,2.18,0.88,MON,2018-06-26,127.9500,126.82
3,After Market Close,2.01,0.64,5.30,NKE,2018-06-27,75.0300,71.05
4,After Market Close,4.12,3.12,10.27,MU,2018-06-27,59.9701,53.81


In [7]:
vols

,daily_ann,intra_ann,ovrnt_ann,close,daily_dollar_vol
IQ,0.807808,1.575152,0.323264,40.16,2.061436
TAL,0.523635,1.064833,0.228215,38.74,1.335270
DBX,0.603286,1.042362,0.133687,39.60,1.372683
WLL,0.521462,0.981390,0.250578,46.78,1.696323
GBT,0.664695,0.966065,0.259535,40.05,1.970072
OLED,0.406104,0.962741,0.695005,93.95,2.522398
ROKU,0.500241,0.928412,0.233704,43.30,1.418680
NAV,0.425583,0.843755,0.235968,42.83,1.050116
THO,0.344307,0.787095,0.289389,98.69,2.136399
M,0.427123,0.773007,0.089697,38.27,1.010597


In [133]:
calls, puts = greek_calc('TQQQ', 45, 20, prem_price_use = 'Mid')

C:\Users\Fang\Anaconda3\lib\site-packages\py_vollib\ref_python\black_scholes_merton\__init__.py:87: RuntimeWarning: divide by zero encountered in double_scalars
  return numerator / denominator
C:\Users\Fang\Anaconda3\lib\site-packages\py_vollib\black_scholes_merton\greeks\analytical.py:210: RuntimeWarning: invalid value encountered in double_scalars
  return numerator / denominator


In [141]:
price_sim(calls, 0.05, 0.05, 10)

C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Fang\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,Strike,Expiry,DTE,Type,Last,Bid,Ask,Mid,Underlying_Price,Simulated Price,Price Change
0,54.0,2018-07-13,26,call,9.50,9.20,9.50,9.350,62.74,12.031160,0.286755
1,56.5,2018-07-13,26,call,6.60,6.20,6.60,6.400,62.74,9.395967,0.468120
2,57.0,2018-07-13,26,call,7.60,6.70,6.90,6.800,62.74,9.208503,0.354192
3,57.5,2018-07-13,26,call,6.50,5.40,5.80,5.600,62.74,8.439017,0.506967
4,59.0,2018-07-13,26,call,4.20,4.30,4.70,4.500,62.74,7.061464,0.569214
5,59.5,2018-07-13,26,call,4.38,4.00,4.30,4.150,62.74,6.616127,0.594247
6,60.0,2018-07-13,26,call,3.40,3.70,4.00,3.850,62.74,6.197265,0.609679
7,60.5,2018-07-13,26,call,4.10,4.00,4.20,4.100,62.74,6.089608,0.485270
8,61.0,2018-07-13,26,call,3.99,3.60,3.90,3.750,62.74,5.668156,0.511508
9,61.5,2018-07-13,26,call,2.35,2.80,3.10,2.950,62.74,4.972691,0.685658


In [140]:
calls

,Strike,Expiry,Type,Last,Bid,Ask,Vol,Open_Int,IV,Underlying_Price,DTE,Mid,Calc IV,Delta,Gamma,Theta,Vega,Calc Price
0,54.0,2018-07-13,call,9.50,9.20,9.50,20.0,3.0,0.510747,62.74,26,9.350,0.530093,0.870993,0.023706,-0.035926,0.035235,12.031160
1,56.5,2018-07-13,call,6.60,6.20,6.60,20.0,30.0,0.335456,62.74,26,6.400,0.278899,0.925717,0.030089,-0.012627,0.023530,9.395967
2,57.0,2018-07-13,call,7.60,6.70,6.90,40.0,20.0,0.492925,62.74,26,6.800,0.491248,0.787387,0.035290,-0.045928,0.048610,9.208503
3,57.5,2018-07-13,call,6.50,5.40,5.80,70.0,30.0,0.344245,62.74,26,5.600,0.306413,0.865936,0.042114,-0.021328,0.036183,8.439017
4,59.0,2018-07-13,call,4.20,4.30,4.70,20.0,9.0,0.353522,62.74,26,4.500,0.328384,0.771903,0.054964,-0.031966,0.050609,7.061464
5,59.5,2018-07-13,call,4.38,4.00,4.30,1.0,1.0,0.345221,62.74,26,4.150,0.331036,0.740325,0.058478,-0.034561,0.054280,6.616127
6,60.0,2018-07-13,call,3.40,3.70,4.00,20.0,11.0,0.351569,62.74,26,3.850,0.339229,0.704891,0.060752,-0.037703,0.057786,6.197265
7,60.5,2018-07-13,call,4.10,4.00,4.20,12.0,1.0,0.432867,62.74,26,4.100,0.433216,0.645156,0.051313,-0.051933,0.062330,6.089608
8,61.0,2018-07-13,call,3.99,3.60,3.90,3.0,6.0,0.431646,62.74,26,3.750,0.424395,0.619806,0.053587,-0.052048,0.063767,5.668156
9,61.5,2018-07-13,call,2.35,2.80,3.10,5.0,5.0,0.354010,62.74,26,2.950,0.344288,0.603811,0.066843,-0.042728,0.064528,4.972691


In [142]:
puts

,Strike,Expiry,Type,Last,Bid,Ask,Vol,Open_Int,IV,Underlying_Price,DTE,Mid,Calc IV,Delta,Gamma,Theta,Vega,Calc Price
0,57.0,2018-07-13,put,2.30,1.80,2.05,3.0,12.0,0.633793,62.74,26,1.925,0.657619,-0.262896,0.029624,-0.069079,0.054625,0.788796
1,58.0,2018-07-13,put,1.25,1.20,1.30,4.0,13.0,0.467290,62.74,26,1.250,0.475420,-0.247438,0.039699,-0.048381,0.052921,0.415419
2,58.5,2018-07-13,put,1.35,1.30,1.40,27.0,0.0,0.458013,62.74,26,1.350,0.466266,-0.266139,0.042043,-0.049284,0.054967,0.461140
3,59.0,2018-07-13,put,1.71,1.95,2.10,5.0,1.0,0.534184,62.74,26,2.025,0.554493,-0.312305,0.038118,-0.063194,0.059265,0.834213
4,59.5,2018-07-13,put,2.86,2.55,2.85,6.0,6.0,0.610844,62.74,26,2.700,0.633825,-0.345299,0.034726,-0.075221,0.061715,1.254903
5,60.0,2018-07-13,put,2.00,1.65,1.85,3.0,34.0,0.446295,62.74,26,1.750,0.446849,-0.332113,0.048523,-0.052241,0.060796,0.658310
6,60.5,2018-07-13,put,2.00,2.45,2.65,5.0,3.0,0.523198,62.74,26,2.550,0.542899,-0.373203,0.041648,-0.066188,0.063400,1.145395
7,61.0,2018-07-13,put,2.30,1.90,2.15,10.0,1.0,0.430181,62.74,26,2.025,0.426781,-0.380601,0.053305,-0.052349,0.063788,0.800385
8,61.5,2018-07-13,put,2.55,2.15,2.30,12.0,10.0,0.419439,62.74,26,2.225,0.423800,-0.407854,0.054710,-0.052982,0.065013,0.912122
9,62.0,2018-07-13,put,2.79,2.25,2.50,2.0,8.0,0.414557,62.74,26,2.375,0.411173,-0.435251,0.057178,-0.052121,0.065921,0.993283
